In [1]:
import numpy as np
from random import Random
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from seeds import known_seeds
import evaluation
import utils
from timer import Timer             
from action import ActionSpace

In [2]:
space = spaces.Box(low=0, high=1000000, shape=(6, 7), dtype=np.int32)
space.sample()

array([[357061, 379221, 784334,  80813, 563364, 369763, 167307],
       [575632, 338680, 310030, 240281, 229652, 765678, 332273],
       [148921, 854619,  52248, 669844, 962291, 751035, 398918],
       [156864, 824345,  37104, 367984, 231940,  13493, 702567],
       [ 61636, 488121, 518909, 492454, 953635, 829463, 288541],
       [278456, 320990,   8553, 751406,  91872, 619630, 177110]],
      dtype=int32)

In [3]:
#old fleet function
def init_fleet(self):
    fleet = {"DC1":{},"DC2":{},"DC3":{},"DC4":{}}
    for i in fleet.keys():
        for j in self.server_generations:
            fleet[i].update(j, {})
            fleet[i][j].update("servers", [])
            fleet[i][j].update("timestep_bought", [])
            fleet[i][j].update("total_owned", 0)
    return fleet

In [4]:

action_space = spaces.Box(low=0, high=3, shape=(4,7), dtype=np.int32)
action_space.sample()

array([[0, 2, 1, 2, 0, 0, 1],
       [0, 2, 1, 2, 1, 0, 2],
       [3, 0, 1, 0, 2, 2, 1],
       [2, 0, 2, 0, 2, 1, 1]], dtype=int32)

In [5]:
from gymnasium.wrappers import FlattenObservation # type: ignore
class CustomEnv(gym.Env):
    def __init__(self):
        #super(self).__init__()

        """
        define self.actionspace and self.observation_space below using 
        variables available in "gym.spaces"
        """
        
        print("init")
        #agent action space (actions it can make)
        #datacenter, servergen, action, where the index corresponds to an action and chosen number is the fraction of datacenter to fill with that servergen
        self.action_space = spaces.Box(low=0, high=1, shape=(4,7,4), dtype=np.float32)
        
        self.default_demand, self.datacenters, self.servers, self.selling_prices = utils.load_problem_data()

        self.fleet_columns = ['datacenter_id', 'server_generation', 'server_id', 'action',
       'server_type', 'release_time', 'purchase_price', 'slots_size', 'energy_consumption', 
       'capacity', 'life_expectancy', 'cost_of_moving','average_maintenance_fee', 'cost_of_energy',
       'latency_sensitivity', 'slots_capacity', 'selling_price', 'lifespan', 'moved']

        #agent observation space (what the agent can "see"/information that is fed to agent)
        #a 3d array of latency, server_gen, demand, concatenated with 
        #a 3d array of latency, server_gen, supply
        self.observation_space = spaces.Box(low=0, high=1000000, shape=(6, 7), dtype=np.int32)

        self.seeds_array = known_seeds("training")
        self.seed_counter = 0

        self.server_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']
        self.latencies = ['low', 'medium', 'high']
        self.data_centers = ['DC1', 'DC2', 'DC3', 'DC4']
        
        self.timer = Timer()

    #might need func below to convert agent action into a relevant action
    #def conv_agent_action_to_move(self, action):
    
    #returns mask for the action space based on possible plays
    #def valid_action_mask(self):
    
    #initiallise/reset all of the base variables at the end of the "game"
    #has to return a base/initial observation

    def convert_demand_to_observation(self, demand):
        self.timer.reset()
        demand_observation = np.zeros((3,7), np.int32)
        for i in range(len(self.server_generations)):
            servergen_demand = demand[demand["server_generation"] == self.server_generations[i]]
            for j in range(len(self.latencies)):
                latency_demand = servergen_demand[self.latencies[j]]
                demand_observation[j][i] = latency_demand.sum()
        self.timer.time("convert_demand_to_observation")
        return demand_observation

    def convert_fleet_to_observation(self, fleet):
        self.timer.reset()
        observed_fleet = np.zeros((3,7), np.int32)
        for i in range(len(self.data_centers[0:3])):
            for j in range(len(self.server_generations)):
                #filter for the datacenter
                filtered_dc = fleet[fleet["datacenter_id"] == self.data_centers[i]]
                #get sum of the server generation
                gen_total = filtered_dc[filtered_dc["server_generation"] == self.server_generations[j]].shape[0]
                if(self.server_generations[i] == "DC3"):
                    #get dc4 and add onto dc3 total
                    filtered_dc = fleet[fleet["datacenter_id"] == self.datacenters[i+1]]
                    gen_total += filtered_dc[filtered_dc["server_generation"] == self.server_generations[j]].shape[0]
                observed_fleet[i][j] = gen_total
        self.timer.time("convert_fleet_to_observation")
        return observed_fleet

    def init_fleet(self):
        self.timer.reset()
        self.fleet = pd.DataFrame(columns=self.fleet_columns)
        self.timer.time("init_fleet")

    def reset(self, seed=None, options=None):
        self.timer.reset()
        self.timestep = 1
        self.seed_counter += 1
        self.seed_counter %= 10
        np.random.seed(self.seeds_array[self.seed_counter])

        self.actionspace = ActionSpace()
        self.OBJECTIVE = 0

        self.selling_prices2 = evaluation.change_selling_prices_format(self.selling_prices)
        self.init_fleet()
        self.demand = evaluation.get_actual_demand(self.default_demand)
        self.timestep_demand = self.demand[self.demand["time_step"] == self.timestep]
        observation_demand = self.convert_demand_to_observation(self.timestep_demand)
        observation_fleet = self.convert_fleet_to_observation(self.fleet)
        observation = np.concatenate((observation_demand, observation_fleet))
        self.done = False
        self.timer.time("reset")
        return observation, {}
    
    #caps the buy number to datacenter capacity
    def cap_buy_num(self, datacenter, server_gen, number):
        slots = self.fleet.groupby(by=['datacenter_id']).agg({'slots_size': 'sum',
                                                        'slots_capacity': 'mean'})
        if(slots.empty):
            return number
        server_slotsize = (self.servers[self.servers["server_generation"] == server_gen]["slots_size"]).iloc[0]
        if(server_slotsize * number + slots["slots_size"].mean() > slots["slots_capacity"].mean()):
            number = slots["slots_capacity"].mean() - slots["slots_size"].mean()
        return int(number)

    #buys "number" amount of servers at datacenter
    def buy(self, datacenter, server_gen, number=10):
        self.timer.reset()
        ts_fleet = pd.DataFrame(columns=self.fleet_columns[0:3])
        number = self.cap_buy_num(datacenter, server_gen, number)

        fleet_array = []
        datacenter_array =[]
        server_gen_array=[]
        buy_array=[]
        server_id_array=[]
        for i in range(number):
            server_id = self.actionspace.generate_server_id(server_gen)
            datacenter_array.append(datacenter)
            server_gen_array.append(server_gen)
            server_id_array.append(server_id)
            buy_array.append("buy")
            fleet_array.append([datacenter, server_gen, server_id, "buy"])
        temp = pd.DataFrame({"datacenter_id":datacenter_array, "server_generation":server_gen_array,
         "server_id": server_id_array, "action": buy_array})


        ts_fleet = pd.concat([ts_fleet, temp])
        ts_fleet = ts_fleet.merge(self.servers, on='server_generation', how='left')
        ts_fleet = ts_fleet.merge(self.datacenters, on='datacenter_id', how='left')
        ts_fleet = ts_fleet.merge(self.selling_prices, 
                            on=['server_generation', 'latency_sensitivity'], 
                            how='left')
        ts_fleet = ts_fleet.fillna(0)
        self.fleet = pd.concat([self.fleet, ts_fleet])
        #self.fleet = self.fleet.fillna(0)
        self.timer.time("buy")

    #called in a loop where each time it is called the agent chooses an action and change
    #state of game appropriately according to agent action
    def step(self, action):
        """
        after agent move, do it, calc the new observation state
        and the reward from that move it made, if "end" of game set self.done to True

        """
        reward = 0
        actions = self.actionspace.convert_actionspace_to_actionV2(action)

        for i in actions:
            if(i[0] == "buy"):
                self.buy(datacenter = i[3], server_gen = i[1], number = i[2])

        self.fleet = evaluation.update_check_lifespan(self.fleet)
        Zf = evaluation.get_capacity_by_server_generation_latency_sensitivity(self.fleet)
        D = evaluation.get_time_step_demand(self.demand, self.timestep)
        U = evaluation.get_utilization(D, Zf)
        #check if fleet is empty
        if self.fleet.shape[0] > 0:
            # get the server capacity at timestep
            Zf = evaluation.get_capacity_by_server_generation_latency_sensitivity(self.fleet)

            # evaluate objective function at current timestep
            U = evaluation.get_utilization(D, Zf)

            L = evaluation.get_normalized_lifespan(self.fleet)
    
            P = evaluation.get_profit(D, 
                           Zf, 
                           self.selling_prices2,
                           self.fleet)
                           
            o = U * L * P

            self.OBJECTIVE += o
            reward += U+L
            if(L <= 0):
                reward -=2
        else:
            reward -= 10
        #reached final timestep
        if(self.timestep >= 10):
            self.done = True
        self.timestep += 1

        #extra info on the game if wanted for yourself
        info = {}
        truncated = False
        
        self.timestep_demand = self.demand[self.demand["time_step"] == self.timestep]
        
        observation_demand = self.convert_demand_to_observation(self.timestep_demand)
        observation_fleet = self.convert_fleet_to_observation(self.fleet)
        observation = np.concatenate((observation_demand, observation_fleet))

        return (observation, reward, self.done, truncated, info)


In [6]:
from stable_baselines3.common.env_checker import check_env
env = CustomEnv()
check_env(env)

init
1.8ms (init_fleet)
2.6ms (convert_demand_to_observation)
5.8ms (convert_fleet_to_observation)
0.0ms (reset)
0.9ms (init_fleet)
2.9ms (convert_demand_to_observation)
7.6ms (convert_fleet_to_observation)
0.1ms (reset)
31.3ms (buy)


/home/degen/code/Hackathon-Huawei/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/degen/code/Hackathon-Huawei/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.

142.3ms (buy)
38.8ms (buy)
32.3ms (buy)
29.1ms (buy)
29.9ms (buy)
37.7ms (buy)
34.1ms (buy)
34.1ms (buy)
35.9ms (buy)
40.7ms (buy)
32.7ms (buy)
34.4ms (buy)
32.2ms (buy)
41.6ms (buy)
44.0ms (buy)
36.9ms (buy)
28.8ms (buy)
30.5ms (buy)
31.4ms (buy)
31.6ms (buy)
30.9ms (buy)
29.3ms (buy)
30.8ms (buy)
31.5ms (buy)
30.0ms (buy)
31.7ms (buy)
32.8ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


4.5ms (convert_demand_to_observation)
129.4ms (convert_fleet_to_observation)
2.5ms (init_fleet)
3.2ms (convert_demand_to_observation)
7.7ms (convert_fleet_to_observation)
0.1ms (reset)


/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.concat([self.fleet, ts_fleet])


42.1ms (buy)
36.3ms (buy)
39.2ms (buy)
29.8ms (buy)
31.5ms (buy)
29.9ms (buy)
32.2ms (buy)
35.7ms (buy)
39.5ms (buy)
65.1ms (buy)
40.2ms (buy)
39.0ms (buy)
36.6ms (buy)
44.1ms (buy)
37.0ms (buy)
35.4ms (buy)
32.6ms (buy)
35.7ms (buy)
85.6ms (buy)
37.3ms (buy)
40.3ms (buy)
34.7ms (buy)
35.2ms (buy)
33.9ms (buy)
31.1ms (buy)
32.9ms (buy)
27.9ms (buy)
32.5ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


4.2ms (convert_demand_to_observation)
79.4ms (convert_fleet_to_observation)
20.6ms (buy)
20.2ms (buy)
25.3ms (buy)
20.1ms (buy)
21.7ms (buy)
21.5ms (buy)
24.8ms (buy)
22.9ms (buy)
18.5ms (buy)
19.8ms (buy)
25.3ms (buy)
20.9ms (buy)
22.4ms (buy)
22.4ms (buy)
21.6ms (buy)
26.9ms (buy)
22.8ms (buy)
24.2ms (buy)
23.8ms (buy)
18.9ms (buy)
21.8ms (buy)
18.9ms (buy)
19.1ms (buy)
23.0ms (buy)
21.5ms (buy)
33.1ms (buy)
24.3ms (buy)
21.5ms (buy)
3.6ms (convert_demand_to_observation)
83.3ms (convert_fleet_to_observation)
22.4ms (buy)
21.1ms (buy)
27.6ms (buy)
25.4ms (buy)
29.0ms (buy)
22.2ms (buy)
21.5ms (buy)
24.6ms (buy)
20.8ms (buy)
22.0ms (buy)
23.4ms (buy)
20.5ms (buy)
22.9ms (buy)
21.8ms (buy)
22.6ms (buy)
23.4ms (buy)
23.2ms (buy)
25.1ms (buy)
20.4ms (buy)
30.8ms (buy)
18.7ms (buy)
18.1ms (buy)
23.7ms (buy)
23.5ms (buy)
24.7ms (buy)
22.5ms (buy)
22.0ms (buy)
25.1ms (buy)
2.7ms (convert_demand_to_observation)
73.2ms (convert_fleet_to_observation)
18.2ms (buy)
21.0ms (buy)
17.9ms (buy)
18.3m

In [7]:
import os
import time
from stable_baselines3 import PPO
def train():
    #directory model and log saved to
    model_dir = f"models/V1/{int(time.time())}/"
    log_dir = f"logs/V1/{int(time.time())}/"

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    env = CustomEnv()
    #wrap for action masking
    #env = ActionMasker(env, mask_fn) 
    env.reset()

    model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_dir)
    #model = PPO(MaskableActorCriticPolicy, env, verbose=1, tensorboard_log=log_dir)

    TIMESTEPS = 1
    #adjust the range below to adjust timesteps it runs for (calc stepcount as max range val * timesteps)
    #saves model every TIMESTEPS number of steps
    for i in range(1,50):
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print("1 cycle done")
        model.save(f"{model_dir}/{TIMESTEPS*i}")

In [8]:
envv = CustomEnv()
#check environmento
check_env(envv)

#extra checks, unocomment to run
episodes = 4

for episode in range(episodes):
    done = False
    obs = envv.reset()
    while not done:
        random_action = envv.action_space.sample()
        obs, reward, done, trunc, info = env.step(random_action)
        #print("obs", obs)
        print('reward',reward)

init
0.8ms (init_fleet)
2.8ms (convert_demand_to_observation)
6.8ms (convert_fleet_to_observation)
0.1ms (reset)
0.7ms (init_fleet)
2.2ms (convert_demand_to_observation)
5.6ms (convert_fleet_to_observation)
0.1ms (reset)
21.8ms (buy)
29.4ms (buy)
25.8ms (buy)
32.9ms (buy)
26.6ms (buy)


/home/degen/code/Hackathon-Huawei/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/degen/code/Hackathon-Huawei/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.

29.6ms (buy)
31.1ms (buy)
27.3ms (buy)
28.9ms (buy)
27.4ms (buy)
26.5ms (buy)
26.5ms (buy)
28.2ms (buy)
30.4ms (buy)
29.1ms (buy)
27.2ms (buy)
24.9ms (buy)
29.8ms (buy)
28.3ms (buy)
24.7ms (buy)
25.3ms (buy)
27.3ms (buy)
27.3ms (buy)
31.1ms (buy)
28.9ms (buy)
30.6ms (buy)
28.5ms (buy)
26.3ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


2.6ms (convert_demand_to_observation)
63.8ms (convert_fleet_to_observation)
1.4ms (init_fleet)
1.9ms (convert_demand_to_observation)
6.8ms (convert_fleet_to_observation)
0.1ms (reset)
24.8ms (buy)
26.2ms (buy)
26.5ms (buy)


...

3.7ms (convert_demand_to_observation)
84.9ms (convert_fleet_to_observation)
18.1ms (buy)
21.7ms (buy)
19.0ms (buy)
20.3ms (buy)
19.0ms (buy)
19.3ms (buy)
21.0ms (buy)
18.1ms (buy)
19.0ms (buy)
19.7ms (buy)
19.7ms (buy)
19.8ms (buy)
21.8ms (buy)
20.1ms (buy)
21.1ms (buy)
19.7ms (buy)
17.9ms (buy)
18.9ms (buy)
19.4ms (buy)
18.3ms (buy)
18.7ms (buy)
18.7ms (buy)
18.4ms (buy)
17.8ms (buy)
20.3ms (buy)
19.3ms (buy)
20.2ms (buy)
24.2ms (buy)
3.8ms (convert_demand_to_observation)
77.4ms (convert_fleet_to_observation)
18.1ms (buy)
19.0ms (buy)
18.5ms (buy)
20.7ms (buy)
19.3ms (buy)
20.2ms (buy)
22.7ms (buy)
19.5ms (buy)
23.7ms (buy)
17.7ms (buy)
18.3ms (buy)
24.5ms (buy)
21.4ms (buy)
22.6ms (buy)
20.8ms (buy)
18.7ms (buy)
22.8ms (buy)
28.4ms (buy)
21.6ms (buy)
21.1ms (buy)
21.0ms (buy)
19.8ms (buy)
17.5ms (buy)
21.3ms (buy)
21.2ms (buy)
21.2ms (buy)
20.2ms (buy)
20.7ms (buy)
2.5ms (convert_demand_to_observation)
70.8ms (convert_fleet_to_observation)
19.3ms (buy)
19.7ms (buy)
17.7ms (buy)
20.3m

/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.concat([self.fleet, ts_fleet])


35.7ms (buy)
26.9ms (buy)
25.9ms (buy)
27.6ms (buy)
30.5ms (buy)
25.9ms (buy)
27.8ms (buy)
23.3ms (buy)
27.7ms (buy)
28.9ms (buy)
32.8ms (buy)
34.0ms (buy)
34.0ms (buy)
30.3ms (buy)
28.8ms (buy)
30.1ms (buy)
29.7ms (buy)
29.6ms (buy)
27.7ms (buy)
31.9ms (buy)
31.1ms (buy)
27.2ms (buy)
23.9ms (buy)
30.5ms (buy)
30.1ms (buy)
30.3ms (buy)
31.0ms (buy)
27.6ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


4.3ms (convert_demand_to_observation)
90.9ms (convert_fleet_to_observation)
reward 0.02155422430658641
24.4ms (buy)
24.6ms (buy)
19.6ms (buy)
21.1ms (buy)
20.4ms (buy)
22.5ms (buy)
23.7ms (buy)
17.5ms (buy)
20.6ms (buy)
17.3ms (buy)
17.8ms (buy)
19.4ms (buy)
24.0ms (buy)
19.0ms (buy)
23.6ms (buy)
19.6ms (buy)
24.1ms (buy)
19.5ms (buy)
18.2ms (buy)
24.7ms (buy)
18.6ms (buy)
20.0ms (buy)
19.4ms (buy)
17.6ms (buy)
19.2ms (buy)
27.3ms (buy)
20.6ms (buy)
21.4ms (buy)
3.5ms (convert_demand_to_observation)
106.1ms (convert_fleet_to_observation)
reward 0.04549535610303586
20.8ms (buy)
25.0ms (buy)
26.6ms (buy)
33.1ms (buy)
23.2ms (buy)
23.8ms (buy)
19.1ms (buy)
21.0ms (buy)
26.7ms (buy)
21.0ms (buy)
20.8ms (buy)
19.2ms (buy)
19.7ms (buy)
25.8ms (buy)
22.4ms (buy)
17.8ms (buy)
21.5ms (buy)
19.4ms (buy)
22.0ms (buy)
19.0ms (buy)
16.8ms (buy)
17.8ms (buy)
19.6ms (buy)
21.7ms (buy)
26.6ms (buy)
20.2ms (buy)
19.6ms (buy)
20.1ms (buy)
2.6ms (convert_demand_to_observation)
71.1ms (convert_fleet_to_ob

In [9]:
train()
print("done")

init
0.9ms (init_fleet)
2.6ms (convert_demand_to_observation)
5.7ms (convert_fleet_to_observation)
0.0ms (reset)
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
1.4ms (init_fleet)
2.4ms (convert_demand_to_observation)
5.9ms (convert_fleet_to_observation)
0.1ms (reset)
Logging to logs/V1/1725292065/PPO_0
162.1ms (buy)
64.0ms (buy)
115.2ms (buy)


/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.concat([self.fleet, ts_fleet])


162.4ms (buy)
43.0ms (buy)
41.8ms (buy)
38.7ms (buy)
44.3ms (buy)
39.0ms (buy)
39.3ms (buy)
43.5ms (buy)
33.9ms (buy)
47.2ms (buy)
94.0ms (buy)
58.5ms (buy)
60.1ms (buy)
59.1ms (buy)
55.5ms (buy)
54.6ms (buy)
53.5ms (buy)
58.2ms (buy)
47.1ms (buy)
54.6ms (buy)
43.6ms (buy)
42.0ms (buy)
54.9ms (buy)
54.7ms (buy)
47.3ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


4.1ms (convert_demand_to_observation)
209.6ms (convert_fleet_to_observation)
26.7ms (buy)
28.8ms (buy)
25.1ms (buy)
22.2ms (buy)
29.0ms (buy)
27.0ms (buy)
29.7ms (buy)
25.1ms (buy)
27.0ms (buy)
37.8ms (buy)
32.7ms (buy)
39.4ms (buy)
46.9ms (buy)
34.9ms (buy)
25.3ms (buy)
30.8ms (buy)
23.6ms (buy)
30.5ms (buy)
33.4ms (buy)
44.2ms (buy)
29.0ms (buy)
37.4ms (buy)
26.6ms (buy)
36.3ms (buy)
28.1ms (buy)
31.2ms (buy)
29.3ms (buy)
40.6ms (buy)
4.3ms (convert_demand_to_observation)
125.5ms (convert_fleet_to_observation)
25.0ms (buy)
25.3ms (buy)
41.3ms (buy)
44.9ms (buy)
37.7ms (buy)
25.6ms (buy)
28.2ms (buy)
24.5ms (buy)
25.6ms (buy)
25.0ms (buy)
23.5ms (buy)
29.1ms (buy)
25.3ms (buy)
30.0ms (buy)
22.4ms (buy)
25.7ms (buy)
27.3ms (buy)
28.1ms (buy)
34.4ms (buy)
26.5ms (buy)
31.1ms (buy)
25.1ms (buy)
33.4ms (buy)
25.9ms (buy)
26.2ms (buy)
26.9ms (buy)
28.8ms (buy)
25.9ms (buy)
3.6ms (convert_demand_to_observation)
114.4ms (convert_fleet_to_observation)
27.7ms (buy)
26.1ms (buy)
25.4ms (buy)
26

/tmp/ipykernel_9843/4293506490.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.fleet = pd.concat([self.fleet, ts_fleet])


24.1ms (buy)
17.9ms (buy)
65.6ms (buy)
37.7ms (buy)
40.8ms (buy)
49.6ms (buy)
59.1ms (buy)
53.7ms (buy)
38.9ms (buy)
41.5ms (buy)
41.9ms (buy)
41.2ms (buy)
46.8ms (buy)
48.1ms (buy)
44.6ms (buy)
43.2ms (buy)
48.4ms (buy)
41.9ms (buy)
39.1ms (buy)
40.4ms (buy)
38.0ms (buy)
39.9ms (buy)
45.6ms (buy)
40.6ms (buy)
39.7ms (buy)
43.4ms (buy)
43.4ms (buy)
67.6ms (buy)


/home/degen/code/Hackathon-Huawei/rl_agent/evaluation.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fleet['lifespan'] = fleet['lifespan'].fillna(0)


2.7ms (convert_demand_to_observation)
118.4ms (convert_fleet_to_observation)
26.6ms (buy)
31.4ms (buy)
28.2ms (buy)
28.4ms (buy)
32.0ms (buy)
28.9ms (buy)
30.1ms (buy)
25.4ms (buy)
31.0ms (buy)
25.6ms (buy)
30.9ms (buy)
27.5ms (buy)
32.2ms (buy)
31.1ms (buy)
34.2ms (buy)
25.6ms (buy)
30.7ms (buy)
27.4ms (buy)
33.8ms (buy)
28.0ms (buy)
29.0ms (buy)
26.5ms (buy)
27.9ms (buy)
25.1ms (buy)
26.2ms (buy)
27.5ms (buy)
26.0ms (buy)
28.7ms (buy)
2.6ms (convert_demand_to_observation)
119.4ms (convert_fleet_to_observation)
28.2ms (buy)
29.5ms (buy)
27.0ms (buy)
37.3ms (buy)
33.9ms (buy)
27.6ms (buy)
28.9ms (buy)
28.5ms (buy)
30.4ms (buy)
61.0ms (buy)
44.1ms (buy)
27.6ms (buy)
32.8ms (buy)
25.8ms (buy)
30.7ms (buy)
25.6ms (buy)
32.0ms (buy)
32.7ms (buy)
35.9ms (buy)
25.5ms (buy)
29.5ms (buy)
28.1ms (buy)
32.7ms (buy)
28.5ms (buy)
38.5ms (buy)
24.5ms (buy)
26.4ms (buy)
26.4ms (buy)
2.3ms (convert_demand_to_observation)
128.6ms (convert_fleet_to_observation)


KeyboardInterrupt: 